### Importing some important Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
import os
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
#from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

#### Import Data

In [ ]:
train = pd.read_csv('../input/bigmart-sales-data/Train.csv')
test = pd.read_csv('../input/bigmart-sales-data/Test.csv')

#### Data type and check for null values

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

#### Data Clearning and Preprocessing

In [ ]:
train['Item_Weight'].mean()

In [ ]:
train['Item_Weight']=train['Item_Weight'].fillna(train['Item_Weight'].mean())

In [ ]:
train.isnull().sum()

In [ ]:
test['Item_Weight']=test['Item_Weight'].fillna(test['Item_Weight'].mean())

In [ ]:
test.isnull().sum()

In [ ]:
train['Outlet_Size']

In [ ]:
train.head()

In [ ]:
train.loc[train['Outlet_Type']=='Grocery Store','Outlet_Size'] = 'Small'

In [ ]:
train.isnull().sum()

In [ ]:
moda = train['Outlet_Size'].mode()[0]

In [ ]:
train['Outlet_Size'] = train['Outlet_Size'].fillna(moda)

In [ ]:
train.isnull().sum()

In [ ]:
train['Outlet_Size'].value_counts()

In [ ]:
test.loc[test['Outlet_Type']=='Grocery Store','Outlet_Size'] = 'Small'

In [ ]:
moda = test['Outlet_Size'].mode()[0]
test['Outlet_Size'] = test['Outlet_Size'].fillna(moda)

In [ ]:
test['Outlet_Size'].isnull().sum()

In [ ]:
test.isna().sum()

In [ ]:
train.head()

In [ ]:
for col in train.columns:
    print('Value Count is: ',train[col].value_counts())

In [ ]:
test['Item_Fat_Content'] = test['Item_Fat_Content'].replace({'LOW FAT':'Low Fat','LF':'Low Fat','Regular':'Regular','reg':'Regular','low fat':'Low Fat'})

In [ ]:
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace({'LOW FAT':'Low Fat','LF':'Low Fat','Regular':'Regular','reg':'Regular','low fat':'Low Fat'})

In [ ]:
train.head()

In [ ]:
train['Item_Identifier'] = train['Item_Identifier'].astype(str).str[:2]

In [ ]:
train.head()

In [ ]:
train.loc[train['Item_Identifier']=='NC','Item_Fat_Content'] = 'Non-Edible'

In [ ]:
test['Item_Identifier'] = test['Item_Identifier'].astype(str).str[:2]

In [ ]:
test.head()

In [ ]:
print(test.shape)
train.shape

In [ ]:
test.loc[test['Item_Identifier']=='NC','Item_Fat_Content'] = 'Non-Edible'

In [ ]:
train['Item_Fat_Content'].value_counts()

In [ ]:
test['Item_Fat_Content'].value_counts()

#### EDA

In [ ]:
test.groupby(['Item_Type'])['Item_Fat_Content'].count().sort_values().plot.bar()

In [ ]:
train.groupby(['Item_Type'])['Item_Fat_Content'].count().sort_values().plot.bar()

In [ ]:
train.groupby(['Item_Type'])['Item_Visibility'].count().sort_values().plot.bar()

In [ ]:
test.groupby(['Item_Type'])['Item_Visibility'].count().sort_values().plot.bar()

In [ ]:
sns.distplot(train['Item_MRP'],bins=30)

In [ ]:
sns.catplot(x='Outlet_Size',hue='Item_Fat_Content',data=train,kind='count')

In [ ]:
sns.catplot(x='Outlet_Establishment_Year',hue='Outlet_Size',data=train,kind='count')

#### Converting categorical values to numeric values

In [ ]:
train['Outlet_Establishment_Year'].max()

In [ ]:
test['Outlet_Establishment_Year'].max()

In [ ]:
train['Outlet_Establishment_Year'] = 2010-train['Outlet_Establishment_Year']

In [ ]:
test['Outlet_Establishment_Year'] = 2010-test['Outlet_Establishment_Year']

In [ ]:
train.head()

In [ ]:
train=train.drop(['Item_Identifier','Outlet_Identifier'],axis=1)

In [ ]:
train.shape

In [ ]:
test=test.drop(['Item_Identifier','Outlet_Identifier'],axis=1)

In [ ]:
test.shape

In [ ]:
dtest = pd.get_dummies(test)

In [ ]:
dtest.head()

In [ ]:
dtrain = pd.get_dummies(train)

In [ ]:
dtrain.head()

In [ ]:
y=dtrain['Item_Outlet_Sales']

In [ ]:
x=dtrain.drop(['Item_Outlet_Sales'],axis=1)
xx=x.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
stds=StandardScaler()
x = stds.fit_transform(x)
#x['Item_Weight']=stds.fit_transform(np.array(x['Item_Weight']).reshape(-1,1))

In [ ]:
x=pd.DataFrame(x,columns=xx.columns)

In [ ]:
dtestt=dtest.copy()
dtest = stds.fit_transform(dtest)
dtest = pd.DataFrame(dtest,columns=dtestt.columns)

In [ ]:
dtest.head()

### Preparing Data for Machine Learning Model (Train, Test, Split)

In [ ]:
XX_train,X_test,yy_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
X_train,X_cv,y_train,y_cv = train_test_split(XX_train,yy_train, test_size=0.2,random_state=42)

### Random-Forest Regressor

In [ ]:
RF = RandomForestRegressor()
RF_est = {'n_estimators':range(10,1500,100),'max_depth':range(1,50)}
RF_model = RandomizedSearchCV(RF,RF_est,scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
RF_model.fit(X_cv,y_cv)

In [ ]:
print(RF_model.best_score_)
print(RF_model.best_estimator_)
print(RF_model.best_params_)


In [ ]:
RF_Result = RandomForestRegressor(n_estimators=810,max_depth=8)
RF_Result.fit(X_train,y_train)
pred1 = RF_Result.predict(X_test)
np.sqrt(mean_squared_error(y_test,pred1))

### Decision-Tree Regressor

In [ ]:
DTR = DecisionTreeRegressor(random_state=42)
DTR_cv = GridSearchCV(DTR,param_grid={'min_samples_split':range(2,20)},scoring='neg_mean_squared_error',cv=5)
DTR_cv.fit(X_cv,y_cv)

In [ ]:
print(DTR_cv.best_score_)
print(DTR_cv.best_estimator_)
print(DTR_cv.best_params_)

In [ ]:
DTR_result = DecisionTreeRegressor(random_state=42,min_samples_split=19)
DTR_result.fit(X_train,y_train)
pred2 = DTR_result.predict(X_test)
np.sqrt(mean_squared_error(pred2,y_test))

### XG-Boost Regressor

In [ ]:
XG = XGBRegressor()
XG.fit(X_train,y_train)
pred3 = XG.predict(X_test)
np.sqrt(mean_squared_error(y_test,pred3))

In [ ]:
X_train.shape

### Stacking 

In [ ]:
from mlxtend.regressor import StackingCVRegressor

In [ ]:
comb = StackingCVRegressor(regressors=(XG,RF,DTR), cv=5,meta_regressor=RF,random_state=42)
comb.fit(X_train,y_train)
X_test.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32']
comb_pred = comb.predict(X_test)
np.sqrt(mean_squared_error(comb_pred,y_test))

In [ ]:
X_test.shape

In [ ]:
dtest.shape

### Predicting the dtest values

In [ ]:
dtest.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32']

final_pred = comb.predict(dtest)

In [ ]:
final_pred